In [1]:
import sys
from pathlib import Path

# Add the 'src' directory to the Python path
sys.path.append(str(Path().resolve().parent / 'src'))
print(sys.path)

['/Users/rhiemh/.rye/py/cpython@3.12.4/lib/python312.zip', '/Users/rhiemh/.rye/py/cpython@3.12.4/lib/python3.12', '/Users/rhiemh/.rye/py/cpython@3.12.4/lib/python3.12/lib-dynload', '', '/Users/rhiemh/workspace.me/sandbox/testmo-automation/.venv/lib/python3.12/site-packages', '/Users/rhiemh/workspace.me/sandbox/testmo-automation/src', '/Users/rhiemh/workspace.me/sandbox/testmo-automation/src']


-------

In [29]:
import sys
import importlib

def reload_all(module_names):
    """
    특정 모듈 및 하위 의존 모듈을 재귀적으로 reload합니다.
    """
    for module_name in list(sys.modules.keys()):
        if any(module_name.startswith(name) for name in module_names):
            try:
                importlib.reload(sys.modules[module_name])
                print(f"Reloaded: {module_name}")
            except Exception as e:
                print(f"Failed to reload {module_name}: {e}")

# 사용 예시: 'runtime' 패키지 및 관련 모듈 재로드
reload_all(["testmo_automation"])

Reloaded: testmo_automation
Reloaded: testmo_automation.runtime
Reloaded: testmo_automation.runtime.asyn
Reloaded: testmo_automation.runtime.asyn.browser
Reloaded: testmo_automation.resovler
Reloaded: testmo_automation.config
Reloaded: testmo_automation.runtime.asyn.pages


In [9]:
from testmo_automation.runtime.asyn import browser
from testmo_automation.runtime.asyn.pages import Login

testmopage = await browser.load_default_page()
login = Login(testmopage)
await login.goto()
await login.login()

In [15]:
await testmopage.context.browser.close()

-----

In [30]:
from testmo_automation.runtime.asyn import browser
from testmo_automation.runtime.asyn.pages import Repository, Login

In [17]:
testmopage = await browser.load_default_page()

In [31]:
repo = Repository(testmopage)
await repo.goto()

In [19]:
await repo.not_expanded_folder_count()

8

In [33]:
await repo.collapse_all_folders()

In [100]:
from pprint import pprint

class TestPage:
    def __init__(self, page):
        self.page = page
        
        self.groups = self.page.locator(selector="div[data-name='groups']")
        self.not_expanded_list = self.groups.locator("[data-name]").locator('[class*="tree__node--expandable"]:not([class*="tree__node--expanded"])')
        self.clickable_to_expand = self.not_expanded_list.first.locator(".tree__node__entry__expansion__expand")
        self.datalist = self.groups.locator('[data-id]')


    async def parse_sub(self):
        ...

    async def printer_data_id_and_name(self, locator, depth):

        for idx in range(await locator.count()):
            target = locator.nth(idx)
            data_id = await target.get_attribute('data-id')
            data_name = await target.get_attribute('data-name')
            depth_symbol = '>' * depth
            print(f"{depth_symbol}data-id={data_id} data-name={data_name}")
                                                 

    async def group_info(self):

        id_structure = dict()
        name_structure = dict()
        
        data_id_map = dict()
        data_components_map = dict()
        
        data_id_parsing_history = dict()
        

        for idx in range(await self.datalist.count()):
            target = self.datalist.nth(idx)
            data_id = await target.get_attribute('data-id')
            data_name = await target.get_attribute('data-name')
            data_id_map[data_id] = data_name
            data_id_parsing_history[data_id] = False
            
        async def runner(data_id, depth):
            target = self.groups.locator(f"div[data-id='{data_id}']")
            subs = target.locator("> .tree__node__subs").locator("> div[data-id]")

            for idx in range(await subs.count()):
                target = subs.nth(idx)
                data_id = await target.get_attribute('data-id')
                data_name = await target.get_attribute('data-name')
                depth_symbol = '>' * depth
                print(f"{depth_symbol}data-id={data_id} data-name={data_name}")
                await runner(data_id, depth+1)
            
        # for data_id in data_id_map.keys():
        depth=1
        await runner(192, depth)

testpage = TestPage(repo.page)
await testpage.group_info()

NameError: name 'locator' is not defined

In [28]:
reload_all(["testmo_automation"])

Reloaded: testmo_automation
Reloaded: testmo_automation.runtime
Reloaded: testmo_automation.runtime.asyn
Reloaded: testmo_automation.runtime.asyn.browser
Reloaded: testmo_automation.resovler
Reloaded: testmo_automation.config
Reloaded: testmo_automation.runtime.asyn.pages


In [316]:
async def get_into_group(page, group_id):
    await page.goto(f"http://{workspace}.testmo.net/repositories/{repository_num}?group_id={group_id}")
    
async def get_cases(page):
    E_testcases = list()
    
    while True:
        E_cases = page.locator(selector="table[data-target='components--table.table']").locator("tbody").locator("tr[data-id]")
        cnt = await E_cases.count()
        print("[count]")
        print(cnt)
        if cnt == 0:
            break
    
        for idx in range(cnt):
            E_tr = E_cases.nth(idx)
            # id = await E_tr.get_attribute("data-id")
            E_testcases.append(E_tr)
            
        print("testcase")
        print(len(E_testcases))
            
        E_next_btn = page.locator(selector="button[data-action='click->components--pagination#doNextPage']")
        is_exist_next = await E_next_btn.is_visible()
        
        print("is_exist_next")
        print(is_exist_next)
        if not is_exist_next:
            break
            
        is_disable_to_click = await E_next_btn.is_disabled()

        print("is disable to click?")
        print(is_disable_to_click)
        if is_disable_to_click:
            break
    
        await E_next_btn.click()
        await asyncio.sleep(0.5)

    print("testcase")
    print(len(E_testcases))        
    return E_testcases

async def get_case_ids(E_cases):
    ids = list()
    cnt = await E_cases.count()
    for idx in range(cnt):
        E_tr = E_cases.nth(idx)
        id = await E_tr.get_attribute("data-id")
        ids.append(id)
    return ids